# Generate clinical plans from patient-physician audio interviews

This notebook demonstrates how to generate clinical plans from patient-physician audio interviews using AWS Managed services and Claude 3 generalised large language model family.  

## Prerequisites
- Verify that model access to Anthropic's Claude 3 Sonnet and Haiku is granted to the account being used, see documentation here: [Amazon Bedrock Model Access](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)

## Instructions
1. The notebook is designed to run with Amazon SageMaker Notebook Instance. For instructions on how to onboard to a Sagemaker Notebook Instances, refer to this [link](https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html).

2. Update your SageMaker IAM role (created when you initially set up the Sagemaker Notebook Instance) to
 contain the following AWS managed policies:

- [AmazonBedrockFullAccess](https://docs.aws.amazon.com/aws-managed-policy/latest/reference/AmazonBedrockFullAccess.html)
- [AmazonTranscribeFullAccess](https://docs.aws.amazon.com/aws-managed-policy/latest/reference/AmazonTranscribeFullAccess.html) 

You can find the SageMaker IAM role attached to your Notebook Instance from the **Amazon SageMaker Console** -> **Notebook Instance** in the section **Permissions and encryption**, as shown below:


## Environment Setup

Update boto3 SDK to version **`1.33.0`** or higher.

In [1]:
!pip install botocore boto3 awscli tscribe pandas ipython --upgrade

  Using cached ipython-8.26.0-py3-none-any.whl.metadata (5.0 kB)
Using cached ipython-8.26.0-py3-none-any.whl (817 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 122.0 MB/s eta 0:00:0000:01
  Attempting uninstall: ipython
    Found existing installation: ipython 8.25.0
    Uninstalling ipython-8.25.0:
      Successfully uninstalled ipython-8.25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hdijupyterutils 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.


## 1. Batch Transcription Using Python SDK

Setting up the environment with the AWS clients and libraries. When setting up a custom name for the sagemaker session bucket, it's essential to start the bucket name with `sagemaker-xxxx`. If not, then the bucket needs to have `S3FullAccess`.

In [52]:
import os
import time
import boto3
import json
import tscribe
import pandas
import datetime
from IPython.display import display_markdown, Markdown, clear_output
import sagemaker

bucket = "sagemaker-physician-patient-interviews"
# create SageMaker session object to interact with SageMaker resources with custom bucket name
sagemaker_session = sagemaker.Session(default_bucket = bucket) 
region = sagemaker_session.boto_session.region_name # get region from boto3 session

s3 = boto3.client('s3', region) # connect to AWS S3 service
transcribe = boto3.client('transcribe', region) # connect to Amazon Transcribe service

#### 1.1. Download the recordings

We will use the sample recording published as part of the supplemental materials of the following paper "Fareez, F., Parikh, T., Wavell, C. et al. A dataset of simulated patient-physician medical interviews with a focus on respiratory cases. Sci Data 9, 313 (2022). https://doi.org/10.1038/s41597-022-01423-1 

In [3]:
!curl -L --output data.zip https://springernature.figshare.com/ndownloader/files/30598530

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  986M  100  986M    0     0  39.5M      0  0:00:24  0:00:24 --:--:-- 39.9M


In [4]:
!unzip -qq -o data.zip

Input the data within the specified bucket, using a prefix. This prefix will create a 'folder' within the bucket. For S3, the concepts of folders doesn't exist, it's a path, however within the UI they show as folders. 

In [54]:
prefix = "rawdata"
inputs = sagemaker_session.upload_data(path="Data", key_prefix=prefix) # upload data to s3 bucket within 'prefix folder'
print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-physician-patient-interviews/rawdata


In the variable below, indicate the name of the recorded session you want to transcribe and summarise, any object is fine:  
- **`[object_name]`**: file name including the extension (e.g. RES0037.mp3)

In [55]:
object_name = "RES0038.mp3"

We will prefill the value of the `[job_name]` variable such to create unique Transcribe jobs.

In [58]:
timestamp = datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S")
media_uri = f"s3://{bucket}/{prefix}/{'Audio Recordings'}/{object_name}"
job_name = "transcribe-%s-%s" % (object_name.split(".")[0],timestamp)

#### 1.2. Starting an AWS Transcribe job
Invoking **`start_transcription_job`** API to start a transcription job. This is different from the `start_medical_scribe_job` API which is specific for medical jobs. **With this API, the output is only transcription and no summary; the summary will be later generated using LLM models.**

In [59]:
response = transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    LanguageCode='en-US', # language code (if known)
    Media={
        'MediaFileUri': str(media_uri)
    },
    OutputBucketName=bucket,
    Settings={
        'ShowSpeakerLabels': True,
        'MaxSpeakerLabels': 2,
        'ChannelIdentification': False
    }
)
print(response)

{'TranscriptionJob': {'TranscriptionJobName': 'transcribe-RES0038-2024-08-11-185736', 'TranscriptionJobStatus': 'IN_PROGRESS', 'LanguageCode': 'en-US', 'Media': {'MediaFileUri': 's3://sagemaker-physician-patient-interviews/rawdata/Audio Recordings/RES0038.mp3'}, 'StartTime': datetime.datetime(2024, 8, 11, 18, 57, 43, 69000, tzinfo=tzlocal()), 'CreationTime': datetime.datetime(2024, 8, 11, 18, 57, 43, 43000, tzinfo=tzlocal()), 'Settings': {'ShowSpeakerLabels': True, 'MaxSpeakerLabels': 2, 'ChannelIdentification': False}}, 'ResponseMetadata': {'RequestId': '4673b4ee-59cb-4ead-89a2-903612c33d7b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4673b4ee-59cb-4ead-89a2-903612c33d7b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '404', 'date': 'Sun, 11 Aug 2024 18:57:42 GMT'}, 'RetryAttempts': 0}}


#### 1.3. Checking job status

The code below will invoke Transcribe **`get_transcription_job`** API to retrieve the status of the job we started in the previous step. If the status is not Completed or Failed, the code waits 5 seconds to retry until the job reaches a final state.

In [60]:
while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(5)

print("Job status: " + status.get('TranscriptionJob').get('TranscriptionJobName'))

start_time = status.get('TranscriptionJob').get('StartTime')
completion_time = status.get('TranscriptionJob').get('CompletionTime')
diff = completion_time - start_time

print("Job duration: " + str(diff))
print("Transcription file: " + status.get('TranscriptionJob').get('Transcript').get('TranscriptFileUri'))

Job status: transcribe-RES0038-2024-08-11-185736
Job duration: 0:00:56.530000
Transcription file: https://s3.us-east-1.amazonaws.com/sagemaker-physician-patient-interviews/transcribe-RES0038-2024-08-11-185736.json


#### 1.4. Analysing the scribe results
The code below will download the **`transcribe.json`** file generated by Transcribe, will parse the file and extract the diarised transcription.

In [61]:
transcription_file = job_name + ".json"

transcription = s3.get_object(Bucket=bucket, Key=transcription_file) # fetch transcription file from the S3 bucket
body = json.loads(transcription['Body'].read())

s3.download_file(bucket, transcription_file, "output.json") # download transcription as output.json, saved within Jupyter directory

In [62]:
tscribe.write("output.json", format="csv", save_as="output.csv") # write output to CSV file with tscribe

desired_width = 600
pandas.set_option('display.width', desired_width) # set pandas display width

# skip first line and add custom header
transcript = pandas.read_csv("output.csv",  names=["line", "start_time", "end_time", "speaker", "comment"], header=None, skiprows=1)
# create list of strings with interactions (line - speaker - comment)
interaction = ["%s, %s: %s" % (segment[0], segment[1],segment[2]) for segment in transcript[['line','speaker', 'comment']].values.tolist()]
transcript

output.csv written in 0.97 seconds.


,line,start_time,end_time,speaker,comment
0,0,00:00:00,00:00:02,spk_0,So what brings you in here today at the Family...
1,1,00:00:03,00:00:11,spk_1,"Uh, I've been, been coughing these last, uh, t..."
2,2,00:00:11,00:00:13,spk_1,I think I got sick there.
3,3,00:00:14,00:00:18,spk_0,Ok. So just the last couple of weeks you've be...
4,4,00:00:19,00:00:24,spk_1,"Yeah. Ever since I, I got back from Mexico. It..."
...,...,...,...,...,...
152,152,00:11:40,00:11:42,spk_1,"Um Nope, that was it?"
153,153,00:11:43,00:12:00,spk_0,"Ok. So based on what we talked about it, it se..."
154,154,00:12:00,00:12:21,spk_0,um it can also be like a viral upper respirato...
155,155,00:12:22,00:12:24,spk_1,That sounds great. Thank you.


In [63]:
interaction

['0, spk_0: So what brings you in here today at the Family Clinic?',
 "1, spk_1: Uh, I've been, been coughing these last, uh, two weeks since I got back from, uh, Mexico.",
 '2, spk_1: I think I got sick there.',
 "3, spk_0: Ok. So just the last couple of weeks you've been coughing?",
 "4, spk_1: Yeah. Ever since I, I got back from Mexico. It's been, yeah, so about, about two weeks.",
 '5, spk_0: Ok. And, uh, is your cough a wet cough or a dry cough?',
 "6, spk_1: It's dry? Yeah, I'm not bringing up any, any sputum.",
 '7, spk_0: Ok. And are you coughing up any blood at all?',
 '8, spk_1: No blood?',
 '9, spk_0: No. Ok.',
 '10, spk_0: And is the cough, uh, constant or does it come and go?',
 '11, spk_1: It,',
 "12, spk_1: it's, um, it comes and goes, I would say, um, sometimes it can be worse. Um,",
 "13, spk_1: the, yeah, depending on, on, on what I'm, what I'm doing. I guess if I'm exercising or, um,",
 "14, spk_1: if it gets, if it's really cold outside it might, um,",
 '15, spk_1: 

---

## 2. Generate clinical notes using Claude model family

### 2.1. Prompt engineering
Claude is trained to be a helpful, honest, and harmless assistant. It is used to speaking in dialogue, and you can instruct it in regular natural language requests as if you were making requests of a human.The quality of the instructions you give Claude can have a large effect on the quality of its outputs, especially for complex tasks. See https://docs.anthropic.com/claude/docs/intro-to-prompting to learn more about prompt engineering.

Structured enterprise-grade prompts may contain the following sections: 
1. **Task context**
1. Tone context
1. Background data, documents, and images
1. **Detailed task description & rules**
1. Examples
1. Conversation history
1. Immediate task description or request
1. Thinking step by step / take a deep breath

In our scenario, we will use a simplified prompt (template) that will instruct the model to generate a structured summary of the transcribed conversation and indicate the lines in the transcript that support each claim. This summary is divided in the following sections: 

1. Chief complaint
1. History of present illness
1. Review of systems
1. Past medical history
1. Assessment
1. Plan
1. Physical examination

In [64]:
prompt = '''You will be reading a transcript of a recorded conversation between a physician and a patient. You will find the conversation within the transcript XML tags. Your goal is to summarise 
it, capture the most significative insights and propose the appropriate action plan under a section named ‘clinical plan’ that includes the following sections: Chief complaint; History of present 
illness; Review of systems; Past medical history; Assessment; Plan; Physical examination. Per each claim you make, you need to indicate which lines of the transcript supports it (please indicate 
only the line numbers within the tag <line></line>).
<transcript>
%s
</transcript>
''' % "\n".join(interaction)
print(prompt)

You will be reading a transcript of a recorded conversation between a physician and a patient. You will find the conversation within the transcript XML tags. Your goal is to summarise 
it, capture the most significative insights and propose the appropriate action plan under a section named ‘clinical plan’ that includes the following sections: Chief complaint; History of present 
illness; Review of systems; Past medical history; Assessment; Plan; Physical examination. Per each claim you make, you need to indicate which lines of the transcript supports it (please indicate 
only the line numbers within the tag <line></line>).
<transcript>
0, spk_0: So what brings you in here today at the Family Clinic?
1, spk_1: Uh, I've been, been coughing these last, uh, two weeks since I got back from, uh, Mexico.
2, spk_1: I think I got sick there.
3, spk_0: Ok. So just the last couple of weeks you've been coughing?
4, spk_1: Yeah. Ever since I, I got back from Mexico. It's been, yeah, so about, about

### 2.2. Payload preparation and model invocation
The new generation of Claude model only support the Messages API, hence we must format the body of our payload in the following way:

In [65]:
accept = 'application/json'
contentType = 'application/json'
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [{
                    "type": "text",
                    "text": prompt,
                }],
            },
        ],
        "temperature": 0
    }
)

In [69]:
region = "us-west-2" # !!! define a region where the models are activated within Amazon Bedrock
bedrock_runtime = boto3.client('bedrock-runtime', region)

#### 2.2.1 Claude 3 Sonnet

The Bedrock service generates the entire summary for the given prompt in a single output, this can be slow if the output contains large amount of tokens.

Below we explore the option how we can use Bedrock to stream the output such that the user could start consuming it as it is being generated by the model. For this Bedrock supports invoke_model_with_response_stream API providing ResponseStream that streams the output in form of chunks.

Instead of generating the entire output, Bedrock sends smaller chunks from the model. This can be displayed in a consumable manner as well.


In [70]:
def teletype_model_response(stream):
    output = []
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if chunk_obj['type'] == 'content_block_delta':
                    text = chunk_obj['delta']['text']
                    clear_output(wait=True)
                    output.append(text)
                    display_markdown(Markdown(''.join(output)))
                    i += 1

We will print the content of the response immediately as the first string is returned 

In [71]:
%%time # print CPU and Wall times for the whole cell
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'

# Uncomment to get response after it has been fully generated
# response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
# response_body = json.loads(response["body"].read())
# completion = response_body["content"][0]["text"]
# print(completion)

# get response in chunks
response = bedrock_runtime.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
teletype_model_response(response.get('body'))


Clinical Plan:

Chief Complaint:
- Persistent dry cough for the past two weeks since returning from Mexico (<line>1</line>, <line>4</line>)

History of Present Illness:
- Dry cough without sputum production (<line>6</line>)
- No coughing up blood (<line>8</line>)
- Cough comes and goes, worsens with exercise, cold weather, and at night (<line>12</line>, <line>13</line>, <line>14</line>, <line>35</line>, <line>52</line>)
- Cough has not been getting worse (<line>19</line>)
- Possible triggers: excessive alcohol consumption and partying in Mexico (<line>23</line>)
- No cold symptoms like runny nose or sore throat (<line>27</line>)
- Coughing fits impair sleep (<line>35</line>)
- Cough relieved by using son's inhaler (<line>38</line>)

Review of Systems:
- No headache, nausea, or vomiting (<line>50</line>)
- Occasional shortness of breath, especially with exertion and when outside (<line>55</line>, <line>69</line>)
- Itchy eyes when outside (<line>56</line>)
- No fevers, chills, or night sweats (<line>64</line>, <line>66</line>)
- No swelling in feet or hands (<line>71</line>)
- No dizziness or heart palpitations (<line>73</line>, <line>75</line>)
- No chest pain (<line>77</line>)
- No changes in bowel movements or urinary patterns (<line>79</line>, <line>81</line>)
- No weight changes or changes in appetite (<line>86</line>, <line>88</line>)
- No known exposure to tuberculosis (<line>91</line>)
- No loss of taste or smell (<line>93</line>)
- Exposure to cats at home (<line>95</line>)

Past Medical History:
- History of eczema in childhood (<line>60</line>, <line>61</line>)
- Possible remote history of asthma in childhood (<line>43</line>, <line>44</line>)
- No other medical conditions like diabetes or high blood pressure (<line>100</line>)
- No current medications (<line>102</line>)
- Allergies to pollen and environmental allergens (<line>105</line>, <line>106</line>)
- Occasional wheezing noticed (<line>108</line>)
- No previous hospitalizations (<line>112</line>)
- No previous surgeries (<line>114</line>)
- Family history of eczema in sister (<line>119</line>)
- No family history of cancers or lung-related conditions (<line>120</line>)

Assessment:
Based on the history and symptoms, the patient's persistent dry cough, worsening with exercise and at night, relief with bronchodilator use, and past history of possible asthma and environmental allergies, suggest a potential diagnosis of asthma or reactive airway disease exacerbated by environmental triggers.

Plan:
- Order pulmonary function tests to evaluate for asthma or reactive airway disease
- Trial of bronchodilator therapy (inhaler) to assess response and symptom relief
- Consider allergy testing to identify potential environmental triggers
- Provide education on asthma management and avoidance of triggers
- Follow-up to monitor response to treatment and adjust management as needed

Physical Examination:
The transcript does not provide details about the physical examination findings. However, a thorough respiratory examination, including auscultation of the lungs for wheezing or other abnormal breath sounds, would be essential in evaluating this patient's condition.

CPU times: user 1.75 s, sys: 363 ms, total: 2.12 s
Wall time: 16.2 s


#### 2.2.2. Claude 3 Haiku

Let's print the response only when it is returned in full using Claude 3 Haiku. The CPU and Wall times should be significanlty lower as Claude 3 Haiku is quicker and more advanced than Claude 3 Sonnet. 

In [72]:
%%time
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'
response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response["body"].read())
completion = response_body["content"][0]["text"]
display_markdown(Markdown(''.join(completion)))

Clinical Plan:

Chief Complaint: Persistent dry cough for the past 2 weeks since returning from Mexico.

History of Present Illness:
- The patient has been experiencing a dry cough for the past 2 weeks since returning from a trip to Mexico (<line>1</line>, <line>2</line>, <line>4</line>).
- The cough is dry and the patient is not producing any sputum (<line>6</line>).
- The cough is intermittent, with it being worse during exercise or in cold weather (<line>13</line>, <line>14</line>, <line>15</line>).
- The cough has been affecting the patient's sleep, waking them up 2-3 times per week (<line>83</line>, <line>84</line>).
- The patient tried using their son's asthma inhaler, which seemed to help with the cough (<line>38</line>).
- The patient has a history of eczema in the past (<line>60</line>, <line>61</line>) and a remote history of possible asthma in childhood (<line>29</line>, <line>41</line>, <line>43</line>, <line>44</line>).

Review of Systems:
- The patient denies any other symptoms such as fever, chills, night sweats, chest pain, shortness of breath, dizziness, or changes in appetite or weight (<line>46</line>-<line>81</line>).
- The patient reports occasional wheezing when coughing (<line>107</line>, <line>108</line>).
- The patient reports their eyes becoming itchy when they are outside, especially when the cough is worse (<line>55</line>, <line>56</line>).

Past Medical History:
- Eczema in childhood (<line>60</line>, <line>61</line>)
- Possible asthma in childhood (<line>29</line>, <line>41</line>, <line>43</line>, <line>44</line>)
- No other significant medical conditions (<line>99</line>, <line>100</line>)

Assessment:
- The patient's symptoms, including the dry cough, intermittent wheezing, and history of eczema and possible childhood asthma, suggest the possibility of asthma or a viral upper respiratory tract infection.

Plan:
1. Perform pulmonary function tests to assess for asthma (<line>154</line>).
2. Trial the use of a bronchodilator (the patient's son's inhaler) to see if it provides symptomatic relief (<line>154</line>).
3. Advise the patient to avoid potential triggers, such as exercise and cold weather, that may worsen the cough (<line>13</line>, <line>14</line>, <line>15</line>).
4. Discuss smoking cessation and provide resources, as the patient recently quit smoking (<line>138</line>, <line>139</line>).
5. Monitor the patient's progress and consider further evaluation or treatment if the cough persists or worsens.

Physical Examination:
A comprehensive physical examination should be performed to assess the patient's respiratory system and rule out any other underlying conditions.

CPU times: user 6.07 ms, sys: 142 μs, total: 6.21 ms
Wall time: 6.28 s
